In [4]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [10]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
while(True):
    ret, frame = cap.read()
    roi = frame[0:500, 0:500]
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.GaussianBlur(gray, (15, 15), 0)
    thresh = cv2.adaptiveThreshold(gray_blur, 255,
                                   cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 5, 1)
    
    kernel = np.ones((3, 3), np.float64)
    closing = cv2.morphologyEx(thresh, 
                               cv2.MORPH_CLOSE,
                               kernel, 
                               iterations=4)
    contours, hierarchy = cv2.findContours(closing, 
                                           cv2.RETR_EXTERNAL,
                                           cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 2000 or area > 5000:
            continue

        if len(cnt) < 5:
            continue

        ellipse = cv2.fitEllipse(cnt)
        # cv2.ellipse(roi, ellipse, (0,255,0), 2)
        
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        imgcontour = cv2.drawContours(roi, [box], 0, (0, 0, 255), 2)
    
    cv2.imshow('result', roi)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
            
cap.release()
cv2.destroyAllWindows()